In [1]:
import datetime
from time import sleep
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from os.path import basename

import json
import requests
import sys
import time
import argparse
import re
import threading
import landsatM2M 

import os
import rasterio
from rasterio.plot import show
from rasterio.mask import mask
from pyproj import Transformer
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

from apscheduler.schedulers.background import BackgroundScheduler

In [4]:
secretsFilePath = 'satHelperSecrets.json'

with open(secretsFilePath) as f:
    secrets = json.loads(f.read())
    
locations = secrets['locations']

In [5]:
def checkWeather(lat,lon):
    url = 'https://api.weather.gov/points/{},{}'
    url = url.format(lat,lon)
    
    #print("Status: ", r.status_code)
    #print("Response content: ", r.text)
    
    stat = 0
    while stat != 200:
        r = requests.get(url)
        stat = r.status_code
        response = json.loads(r.text)
        sleep(2)

    office = str(response['properties']['gridId'])
    gridx = str(response['properties']['gridX'])
    gridy = str(response['properties']['gridY'])

    url = 'https://api.weather.gov/gridpoints/{}/{},{}/forecast' 
    url = url.format(office,gridx,gridy)
    
    stat = 0
    while stat != 200:
        r = requests.get(url)
        stat = r.status_code
        response = json.loads(r.text)
        sleep(2)
    
    return response['properties']['periods']

#

def checkThermal(lat,lon,observerAlt):
    
    apikeyn2yo = secrets['satHelperSecrets']['apikeyn2yo']
    
    satellites = [
        {   
            'name': 'Landsat 8 ',
            'id': 39084,  
            'fov': 90 - (15/2),
            'gsd': 90
        },
        {   
            'name': 'Landsat 9 ',
            'id': 49260,   
            'fov': 90 - (15/2),
            'gsd': 90
        },
#        {   
#            'name': 'ECOSTRESS  ',
#            'id': 25544,   
#            'fov': 90 - (62/2),
#            'gsd': 70
#        },
#        {   
#            'name': 'Sentinel-3A',
#            'id': 25544,   
#            'fov': 90 - (62/2),
#            'gsd': 1000
#        },
#        {   
#            'name': 'VIIRS SUOMI',
#            'id': 25544,   
#            'fov': 90 - (62/2)
#        },
#        {   
#            'name': 'VIIRS NOAA ',
#            'id': 25544,   
#            'fov': 90 - (62/2)
#        },
        
    ]
    
    days = 10 # max 10 day outlook
    
    weather = checkWeather(lat,lon)
    
    passDict = {}
    
    for satellite in satellites:
        # /radiopasses/{id}/{observer_lat}/{observer_lng}/{observer_alt}/{days}/{min_elevation}
        url = 'https://api.n2yo.com/rest/v1/satellite/radiopasses/{}/{}/{}/{}/{}/{}//&apiKey={}'
        url = url.format(satellite['id'],lat,lon,observerAlt,days,satellite['fov'],apikeyn2yo)
        
        stat = 0
        while stat != 200:
            r = requests.get(url)
            stat = r.status_code
            response = json.loads(r.text)
            sleep(2)
        
        for satpass in response['passes']:
            if len(satpass) > 0:
                #which satellite
                printString = satellite['name']
                #printString += ' - Pixel size: '
                #printString += str(satellite['gsd'])
                #printString += ' meters - '
                printString += ' - '
                #datetime
                printString += str(datetime.datetime.fromtimestamp(satpass['startUTC']))
                checkDate = str(datetime.datetime.fromtimestamp(satpass['startUTC'])).split(' ')[0]
                
                for forecast in weather:
                    if forecast['startTime'].split('T')[0] == checkDate:
                        if satpass['startAzCompass'].startswith('S') and (forecast['name'].endswith('night') or forecast['name'].endswith('Night')):
                            printString += ' - '
                            printString += forecast['shortForecast']
                        elif satpass['startAzCompass'].startswith('N') and not (forecast['name'].endswith('night') or forecast['name'].endswith('Night')): 
                            printString += ' - '
                            printString += forecast['shortForecast']
                    else:
                        continue
                        
                passDict[satpass['startUTC']] = printString
            else:
                continue

    passDict = dict(sorted(passDict.items()))
    return passDict

#

def getLandsat(lat,lon,lookback):

    username = secrets['satHelperSecrets']['usgsUsername']
    password = secrets['satHelperSecrets']['usgsPassword'] 
    filetype = 'band'

    serviceUrl = "https://m2m.cr.usgs.gov/api/api/json/stable/"

    payload = {'username' : username, 'password' : password}    
    apiKey = landsatM2M.sendRequest(serviceUrl + "login", payload)    
    print("API Key: " + apiKey, end='\r', flush=True)

    # https://m2m.cr.usgs.gov/api/docs/reference/ #scene-search

    lowerLeftLat = lat - 0.00001
    lowerLeftLon = lon - 0.00001
    upperRightLat = lat + 0.00001
    upperRightLon = lon + 0.00001

    datasetName = 'landsat_ot_c2_l1'

    payload = { 
                'datasetName' : datasetName, # dataset alias
                'maxResults' : lookback, # max results to return
                'startingNumber' : 1, 
                'sceneFilter' : { 
                    'spatialFilter' : {
                        'filterType': 'mbr',
                        'lowerLeft' : {
                            'latitude': lowerLeftLat,
                            'longitude': lowerLeftLon
                        },
                        'upperRight' : {
                            'latitude': upperRightLat,
                            'longitude': upperRightLon 
                        }
                    }
                } # scene filter
              }

    entityIds = []
    results = landsatM2M.sendRequest(serviceUrl + "scene-search", payload, apiKey)  

    for result in results['results']:
        #print(result['entityId'])
        entityIds.append(result['entityId'])

    #print(entityIds)  
    
    

    listId = 'temp_{}_list'.format(datasetName) # customized list id
    payload = {
            "listId": listId,
            'idField' : 'entityId',
            "entityIds": entityIds,
            "datasetName": datasetName
    }

    print("Adding scenes to list...", end='\r', flush=True)
    count = landsatM2M.sendRequest(serviceUrl + "scene-list-add", payload, apiKey)    
    print("Added", count, "scenes", end='\r', flush=True)

    # Get download options
    payload = {
        "listId": listId,
        "datasetName": datasetName
    }

    print("Getting product download options...", end='\r', flush=True)
    products = landsatM2M.sendRequest(serviceUrl + "download-options", payload, apiKey)
    print("Got product download options", end='\r', flush=True)

    tifListID = []
    tifListDisplay = []
    tifListEntity = []
    datasetID = []

    downloads = []

    for product in products:

        subproducts = product['secondaryDownloads']
        for subproduct in subproducts:
            if ( (subproduct['entityId'].split('_')[-2] == 'B10') 
            and (subproduct['entityId'] not in tifListEntity) 
            and (os.path.exists('./landsatM2M/{}'.format(subproduct['displayId'])) == False) ):
                tifListID.append(subproduct['id'])
                tifListDisplay.append(subproduct['displayId'])
                tifListEntity.append(subproduct['entityId'])
                datasetID.append(subproduct['datasetId'])

                downloads.append({"entityId":subproduct["entityId"], "productId":subproduct["id"]})

    print(downloads)

    label = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") 

    # Remove the list
    payload = {
        "listId": listId
    }
    landsatM2M.sendRequest(serviceUrl + "scene-list-remove", payload, apiKey)                

    # Send download-request
    payLoad = {
        "downloads": downloads,
        "label": label,
        'returnAvailable': True
    }

    print(f"Sending download request ...", end='\r', flush=True)
    results = landsatM2M.sendRequest(serviceUrl + "download-request", payLoad, apiKey)
    print(f"Done sending download request", end='\r', flush=True) 
    #print(results)

    ###

    threads = []

    for result in results['availableDownloads']:       
        print(f"Get download url: {result['url']}", end='\r', flush=True )
        landsatM2M.runDownload(threads, result['url'])

    preparingDownloadCount = len(results['preparingDownloads'])
    preparingDownloadIds = []
    if preparingDownloadCount > 0:
        for result in results['preparingDownloads']:  
            preparingDownloadIds.append(result['downloadId'])

        payload = {"label" : label}                
        # Retrieve download urls
        print("Retrieving download urls...", end='\r', flush=True)
        results = landsatM2M.sendRequest(serviceUrl + "download-retrieve", payload, apiKey, False)
        if results != False:
            for result in results['available']:
                if result['downloadId'] in preparingDownloadIds:
                    preparingDownloadIds.remove(result['downloadId'])
                    print(f"Get download url: {result['url']}", end='\r', flush=True )
                    landsatM2M.runDownload(threads, result['url'])

            for result in results['requested']:   
                if result['downloadId'] in preparingDownloadIds:
                    preparingDownloadIds.remove(result['downloadId'])
                    print(f"Get download url: {result['url']}", end='\r', flush=True )
                    landsatM2M.runDownload(threads, result['url'])

        # Don't get all download urls, retrieve again after 30 seconds
        while len(preparingDownloadIds) > 0: 
            print(f"{len(preparingDownloadIds)} downloads are not available yet. Waiting for 30s to retrieve again")
            time.sleep(30)
            results = landsatM2M.sendRequest(serviceUrl + "download-retrieve", payload, apiKey, False)
            if results != False:
                for result in results['available']:                            
                    if result['downloadId'] in preparingDownloadIds:
                        preparingDownloadIds.remove(result['downloadId'])
                        print(f"Get download url: {result['url']}", end='\r', flush=True )
                        landsatM2M.runDownload(threads, result['url'])

    print("Got download urls for all downloads", end='\r', flush=True)                
    # Logout
    endpoint = "logout"  
    if landsatM2M.sendRequest(serviceUrl + endpoint, None, apiKey) == None:        
        print("Logged Out", end='\r', flush=True)
    else:
        print("Logout Failed", end='\r', flush=True)  

    print("Downloading files... Please do not close the program", end='\r', flush=True)
    for thread in threads:
        thread.join()

    print("Complete Downloading", end='\r', flush=True)

#

def convertLatLon(lat,lon,epsgNumber=2263):
    transformer = Transformer.from_crs( "epsg:4326", "epsg:{}".format(epsgNumber) ) 
    x, y = transformer.transform(lat, lon)
    return x, y

#

def friendlyImages(centerNorthSouth,centerEastWest,viewBoxSizes,levelsBoxSizes,locationName):
    
    for tif in os.listdir('landsatM2M/'):
        if tif.endswith('.TIF'):
            
            savename = tif.split('_')[3]
            savename = '{} {} {}'.format(savename[0:4],savename[4:6],savename[6:])
            savename = datetime.datetime.strptime(savename, '%Y %m %d')
            savename = savename.strftime("%Y_%m_%d")
            
            if os.path.exists('pngs/{}.png'.format(savename)) == False:
                fp = r'landsatM2M/{}'.format(tif)
                img = rasterio.open(fp)
                
                plt.ioff()
                plt.clf()
                
                fig = plt.figure(figsize=(16,9), dpi=150, constrained_layout=True)
                
                title = tif.split('_')[3]
                title = '{} {} {}'.format(title[0:4],title[4:6],title[6:])
                title = datetime.datetime.strptime(title, '%Y %m %d')
                title = title.strftime("%d %B %Y")
                title = '{} \n{}'.format(locationName,title)
                
                ######### add more details to the plot title!
                
                plt.title(title)
                plt.axis('off')
                rows = 1
                columns = len(viewBoxSizes)

                for subplot in range(0,len(viewBoxSizes)):

                    north = centerNorthSouth + viewBoxSizes[subplot]
                    south = centerNorthSouth - viewBoxSizes[subplot]
                    east = centerEastWest + viewBoxSizes[subplot]
                    west = centerEastWest - viewBoxSizes[subplot]

                    levelsNorth = centerNorthSouth + levelsBoxSizes[subplot]
                    levelsSouth = centerNorthSouth - levelsBoxSizes[subplot]
                    levelsEast = centerEastWest + levelsBoxSizes[subplot]
                    levelsWest = centerEastWest - levelsBoxSizes[subplot]

                    clipGeom = [{'type': 'Polygon', 'coordinates': [[[west, north],\
                                                          [east, north],\
                                                          [east, south],\
                                                          [west, south],\
                                                          [west, north]]]}]

                    levelsGeom = [{'type': 'Polygon', 'coordinates': [[[levelsWest, levelsNorth],\
                                                          [levelsEast, levelsNorth],\
                                                          [levelsEast, levelsSouth],\
                                                          [levelsWest, levelsSouth],\
                                                          [levelsWest, levelsNorth]]]}]



                    out_img, out_transform = mask(dataset=img, shapes=clipGeom, crop=True)

                    out_meta = img.meta.copy()
                    #print(out_meta)
                    epsg_code = int(img.crs.data['init'][5:])
                    out_meta.update({"driver": "GTiff","height": out_img.shape[1],
                                     "width": out_img.shape[2],"transform": out_transform,"crs": img.crs})

                    out_tif = 'crops/crop_{}'.format(tif)

                    with rasterio.open(out_tif, "w", **out_meta) as dest:
                        dest.write(out_img)

                    clipped = rasterio.open(out_tif)

                    levels_img, levels_transform = mask(dataset=img, shapes=levelsGeom, crop=True)
                    tempMean = np.average(levels_img)
                    tempStd = np.std(levels_img)
                    #print(tempStd)
                    tempLow = np.min(levels_img) - (tempStd*5)
                    tempHigh = np.max(levels_img) + (tempStd*10)

                    fig.add_subplot(rows, columns, subplot+1)
                    plt.axis('off')
                    plt.imshow(out_img.squeeze(), interpolation='nearest',cmap='RdYlBu_r', vmin=tempLow, vmax=tempHigh)
                    #plt.colorbar(fraction=0.046, pad=0.04)

                #plt.show()
                savename = tif.split('_')[3]
                savename = '{} {} {}'.format(savename[0:4],savename[4:6],savename[6:])
                savename = datetime.datetime.strptime(savename, '%Y %m %d')
                savename = savename.strftime("%Y_%m_%d")
                
                plt.savefig('pngs/{}.png'.format(savename),dpi=150)
                plt.clf()
                plt.close()
                #show(img.read(1), transform=img.transform, cmap='RdYlBu')
                #show(clipped.read(1), transform=out_transform, cmap='RdYlBu', adjust = False)

#

def sendEmail(toEmail,subject,message, files=None):
    port_number = 1025
    msg = MIMEMultipart()
    msg['From'] = secrets['satHelperSecrets']['emailUsername']
    msg['To'] = toEmail
    msg['Subject'] = subject
    msg.attach(MIMEText(message))
    
    for f in files or []:
        with open(f, "rb") as fil:
            part = MIMEApplication(
                fil.read(),
                Name=basename(f)
            )
        # After the file is closed
        part['Content-Disposition'] = 'attachment; filename="%s"' % basename(f)
        msg.attach(part)
    
    mailserver = smtplib.SMTP('127.0.0.1',port_number)
    mailserver.login(secrets['satHelperSecrets']['emailUsername'], secrets['satHelperSecrets']['emailPassword'])
    mailserver.sendmail(secrets['satHelperSecrets']['emailUsername'],toEmail,msg.as_string())
    mailserver.quit()
    
#  

def sendNotices(locations):
    for location in locations:
        printText = '{} \n \n'.format(location['name'])
        passDict = checkThermal(location['lat'],location['lon'],location['observerAlt'])
        for item in passDict:
            printText += passDict[item]
            printText += ' \n'
        print(printText)
        for distro in location['distros']:
            sendEmail(distro,'Upcoming thermal image opportunities for {} as of {}'.format(location['name'],
                    datetime.datetime.now().strftime("%Y-%m-%d %H:%M")),printText)
    
    nextTime = list(passDict.items())[0][1].split(' - ')[1]
    nextTime = datetime.datetime.strptime(nextTime, '%Y-%m-%d %H:%M:%S')
    nextTime = nextTime + datetime.timedelta(hours=5)
    nextTime = nextTime.strftime("%Y-%m-%d %H:%M:%S")
    
    #job = sendImagesAndSked(locations,nextTime)
    
    return job
#
        
def sendImages(locations):
    
    for location in locations:
        
        for hour in range(0,24):
            # get recently collected images
            lookback = 5
            getLandsat(location['lat'],location['lon'],lookback)

            # process into friendly images
            friendlyImages(location['centerNorthSouth'], location['centerEastWest'], 
                           location['viewBoxSizes'], location['levelBoxSizes'],location['name'])

            # check for upcoming opportunities as extra info
            printText = '{} Upcoming Images \n \n'.format(location['name'])
            passDict = checkThermal(location['lat'],location['lon'],location['observerAlt'])
            for item in passDict:
                printText += passDict[item]
                printText += ' \n'

            logDF = pd.read_csv('satHelperEmailLog.csv')
            logged = logDF['filePath'].tolist()

            fileList = []
            for png in os.listdir('pngs/'):
                if png.endswith('.png') and png not in logged:
                    fileList.append('./pngs/{}'.format(png))
                    tempdf = pd.DataFrame.from_dict({'filePath':[png],'timeSent':[datetime.datetime.now()]})
                    logDF = pd.concat([logDF,tempdf])

            printText += ' \n'
            printText += 'Please see attached recent image results.'
            printText += ' \n'

            print(printText)

            if len(fileList) > 0:
                
                # send emails to distro list
                for distro in location['distros']:
                    sendEmail(distro,'Thermal image results for {} as of {}'.format(location['name'],
                                datetime.datetime.now().strftime("%Y-%m-%d %H:%M")),printText,files=fileList)
                # log files sent
                logDF[['filePath','timeSent']].to_csv('satHelperEmailLog.csv')
                #schedule next run
                nextTime = list(passDict.items())[0][1].split(' - ')[1]
                nextTime = datetime.datetime.strptime(nextTime, '%Y-%m-%d %H:%M:%S')
                nextTime = nextTime + datetime.timedelta(hours=5)
                nextTime = nextTime.strftime("%Y-%m-%d %H:%M:%S")
                #job = sendImagesAndSked(locations,nextTime)
                #print(job)  
                break

            else:
                sleep(60*60)
        
    return job
            
        


            
def sendImagesAndSked(locations,timeString):
    
    job = scheduler.add_job(sendImages, 'date', 
                            run_date = timeString, 
                            args=[locations], 
                            replace_existing=True)
    
    return job
    

In [ ]:
scheduler = BackgroundScheduler()
scheduler.start()
nextTime = '2023-04-10 09:37:00'
job = sendImagesAndSked(locations,nextTime)

#print(scheduler.get_jobs())
#scheduler.print_jobs()
#scheduler.remove_job('')